In [ ]:
!apt-get install -y curl > /dev/null
!pip install fastapi uvicorn nest-asyncio pyngrok streamlit requests > /dev/null

import os, time, subprocess, threading, requests, nest_asyncio
from pyngrok import ngrok
nest_asyncio.apply()

# --- Cài Ollama ---
print("Cài đặt Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh

print("Khởi động Ollama server...")
ollama_proc = subprocess.Popen(["ollama", "serve"])
time.sleep(10)

print("Tải model gemma:2b (nếu chưa có)...")
!ollama pull gemma:2b || echo "Đã có model gemma:2b"

# --- FastAPI App ---
app_code = r"""
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import List, Dict
import requests, json, time

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

users = {"guest": "1234"}
user_history = {}

class LoginRequest(BaseModel):
    username: str
    password: str

class RegisterRequest(BaseModel):
    username: str
    password: str

class ItineraryRequest(BaseModel):
    username: str
    origin: str
    destination: str
    dates: Dict[str, str]
    interests: List[str]
    pace: str

@app.post("/login")
def login(req: LoginRequest):
    if req.username in users and users[req.username] == req.password:
        return {"ok": True}
    raise HTTPException(status_code=401, detail="Invalid username/password")

@app.post("/register")
def register(req: RegisterRequest):
    if req.username in users:
        raise HTTPException(status_code=400, detail="User already exists")
    users[req.username] = req.password
    user_history[req.username] = []
    return {"ok": True}

def generate_itinerary_from_model(prompt: str):
    url = "http://127.0.0.1:11434/api/generate"
    data = {"model": "gemma:2b", "prompt": prompt}
    try:
        response = requests.post(url, json=data, timeout=180)
        if response.status_code == 200:
            lines = [json.loads(l)["response"] for l in response.text.splitlines() if '"response"' in l]
            return "".join(lines)
        return f"Error {response.status_code}: {response.text[:200]}"
    except Exception as e:
        return f"Error calling Ollama: {e}"

@app.post("/generate_itinerary")
def create_itinerary(req: ItineraryRequest):
    if req.username not in users:
        raise HTTPException(status_code=401, detail="Unknown user")

    prompt = f'''
You are a travel planner AI. Create a day-by-day itinerary from {req.origin} to {req.destination}
from {req.dates.get('start','')} to {req.dates.get('end','')}.
Focus on interests: {', '.join(req.interests)}.
Use pace: {req.pace}.
Divide each day into morning, afternoon, and evening.
'''
    result = generate_itinerary_from_model(prompt)
    entry = {
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "origin": req.origin,
        "destination": req.destination,
        "dates": req.dates,
        "interests": req.interests,
        "pace": req.pace,
        "result": result
    }
    user_history.setdefault(req.username, []).append(entry)
    return {"itinerary": result}

@app.get("/history/{username}")
def get_history(username: str):
    if username not in users:
        raise HTTPException(status_code=401, detail="Unknown user")
    return {"history": user_history.get(username, [])}

@app.post("/history/{username}/clear")
def clear_history(username: str):
    if username not in users:
        raise HTTPException(status_code=401, detail="Unknown user")
    user_history[username] = []
    return {"ok": True}
"""

with open("app.py", "w") as f:
    f.write(app_code)

def run_fastapi():
    os.system("uvicorn app:app --host 0.0.0.0 --port 8000 --log-level warning")

threading.Thread(target=run_fastapi, daemon=True).start()
time.sleep(6)
print("FastAPI server đang chạy.")

# --- Streamlit App ---
streamlit_code = r'''
import streamlit as st
import requests
import json

st.set_page_config(page_title="Travel Planner", layout="wide")
API_BASE = "http://localhost:8000"

if "logged_in" not in st.session_state:
    st.session_state.logged_in = False
if "username" not in st.session_state:
    st.session_state.username = ""

def login_ui():
    st.title("Đăng nhập")
    username = st.text_input("Tên đăng nhập")
    password = st.text_input("Mật khẩu", type="password")
    if st.button("Đăng nhập"):
        try:
            res = requests.post(f"{API_BASE}/login", json={"username": username, "password": password})
            if res.status_code == 200:
                st.session_state.logged_in = True
                st.session_state.username = username
                st.experimental_rerun()
            else:
                st.error("Sai tài khoản hoặc mật khẩu.")
        except Exception as e:
            st.error(f"Lỗi: {e}")

    st.divider()
    st.subheader("Đăng ký tài khoản mới")
    new_user = st.text_input("Tên đăng ký")
    new_pass = st.text_input("Mật khẩu đăng ký", type="password")
    if st.button("Đăng ký"):
        try:
            r = requests.post(f"{API_BASE}/register", json={"username": new_user, "password": new_pass})
            if r.status_code == 200:
                st.success("Đăng ký thành công, vui lòng đăng nhập.")
            else:
                st.error("Tên đã tồn tại.")
        except Exception as e:
            st.error(f"Lỗi: {e}")

def main_ui():
    st.sidebar.write(f"Đang đăng nhập: {st.session_state.username}")
    if st.sidebar.button("Đăng xuất"):
        st.session_state.logged_in = False
        st.experimental_rerun()

    st.title("Travel Itinerary Planner")
    st.write("Nhập thông tin chuyến đi để tạo lịch trình.")

    with st.form("create_form"):
        origin = st.text_input("Điểm khởi hành", "Tokyo")
        destination = st.text_input("Điểm đến", "Kyoto")
        start_date = st.date_input("Ngày bắt đầu")
        end_date = st.date_input("Ngày kết thúc")
        interests = st.multiselect("Sở thích", ["food", "museums", "nature", "nightlife"], ["food", "nature"])
        pace = st.selectbox("Tốc độ hành trình", ["relaxed", "normal", "tight"], index=1)
        submitted = st.form_submit_button("Tạo lịch trình")

    if submitted:
        payload = {
            "username": st.session_state.username,
            "origin": origin,
            "destination": destination,
            "dates": {"start": str(start_date), "end": str(end_date)},
            "interests": interests,
            "pace": pace
        }
        st.info("Đang tạo lịch trình...")
        try:
            r = requests.post(f"{API_BASE}/generate_itinerary", json=payload, timeout=180)
            if r.status_code == 200:
                data = r.json()
                st.text_area("Kết quả", data.get("itinerary", ""), height=300)
            else:
                st.error("Lỗi server.")
        except Exception as e:
            st.error(f"Lỗi kết nối: {e}")

    st.divider()
    st.subheader("Lịch sử của bạn")
    try:
        r = requests.get(f"{API_BASE}/history/{st.session_state.username}")
        if r.status_code == 200:
            history = r.json().get("history", [])
            if not history:
                st.write("Chưa có lịch sử.")
            for item in reversed(history):
                with st.expander(f"{item['origin']} → {item['destination']} ({item['timestamp']})"):
                    st.write(f"Từ {item['dates']['start']} đến {item['dates']['end']}")
                    st.write(f"Sở thích: {', '.join(item['interests'])} - Pace: {item['pace']}")
                    st.text_area("Chi tiết", item["result"], height=200)
            if st.button("Xóa lịch sử"):
                requests.post(f"{API_BASE}/history/{st.session_state.username}/clear")
                st.success("Đã xóa lịch sử.")
                st.experimental_rerun()
    except Exception as e:
        st.error(f"Lỗi: {e}")

if not st.session_state.logged_in:
    login_ui()
else:
    main_ui()
'''

with open("streamlit_app.py", "w") as f:
    f.write(streamlit_code)
print("Streamlit app đã được tạo.")

# --- Chạy Streamlit + ngrok ---
streamlit_proc = subprocess.Popen(
    ["streamlit", "run", "streamlit_app.py", "--server.port", "8501", "--server.address", "0.0.0.0"]
)
time.sleep(8)

ngrok_token = input("Nhập Ngrok Authtoken (https://dashboard.ngrok.com/get-started/your-authtoken): ").strip()
if ngrok_token:
    ngrok.set_auth_token(ngrok_token)
    public_url = ngrok.connect(8501)
    print(f"Truy cập ứng dụng tại: {public_url.public_url}")
else:
    print("Chạy nội bộ tại cổng 8501 (Colab).")
print("Đăng nhập mặc định: guest / 1234")
